In [1]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch
import os
from natsort import natsorted
import json
from tqdm import tqdm
import numpy as np
import regex as re

In [50]:
with open("/data/jiwoong/workspace/MDAgents/output/gpt-4o-mini_medqa_adaptive.json", 'r') as jsfile:
    result = json.load(jsfile)

In [51]:
len(result)

38

In [32]:
import json
result_path = "/data/jiwoong/workspace/MedAgents/outputs/MedQA/50_sampled_hard_medqa/gpt4omini-base_cot"
results = [] 
with open(result_path, 'r') as file:
    for line in file:
        results.append(json.loads(line))

In [33]:
count = 0
for i in results:
    if i['pred_answer'] == i['gold_answer']:
        count += 1
count/50

0.34

In [14]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530"
)

In [15]:
collections = client.list_collections()

In [16]:
collections

['medagents', 'rag2']

In [3]:
if client.has_collection(collection_name="medagents"):
#    client.drop_collection(collection_name="medagents")

In [ ]:
def data_upload(client, collection_name, corpus_list=['statpearls','recop','textbooks','cpg']):
    global_offset = 0
    batch_size = 10000
    client.create_collection(
        collection_name=collection_name,
        dimension=768,
        metric_type="IP",  # Inner product distance
        consistency_level="Strong",  # Strong consistency level
    )
    print(f"data uploading to '{collection_name}' collection in milvus client...\n")
    for corpus_name in corpus_list:
        print(f"Processing corpus: {corpus_name}")
        chunk_numbers = len(os.listdir(f"/data/jiwoong/workspace/milvus/corpus/{corpus_name}/data/"))
        for chunk_num in range(chunk_numbers):
            vectors = np.load(f"/data/jiwoong/workspace/milvus/corpus/{corpus_name}/vector/{corpus_name}_embeds_{chunk_num}.npy")
            with open(f"/data/jiwoong/workspace/milvus/corpus/{corpus_name}/json/{corpus_name}_chunk_{chunk_num}.json", 'r') as jsfile:
                texts = json.load(jsfile)
            data_loaded = [
                    {"vector": vectors[ith].tolist(), "text": texts[ith]['contents_chunk'], "source": corpus_name}
                    for ith in range(0, len(vectors))
                ]
            for i in range(0, len(data_loaded), batch_size):
                batch = data_loaded[i:i+batch_size]
                for idx, item in enumerate(batch):
                    item['id'] = global_offset + idx
                client.insert(collection_name=collection_name, data=batch)
                global_offset += len(batch)
        print(f"{corpus_name} Uploaded!")

In [85]:
def data_upload(client, collection_name='rag2', corpus_list=['statpearls','recop','textbooks','cpg']):
    global_offset = 0
    batch_size = 10000
    client.create_collection(
        collection_name=collection_name,
        dimension=768,
        metric_type="IP",  # Inner product distance
        consistency_level="Strong",  # Strong consistency level
    )
    print(f"data uploading to '{collection_name}' collection in milvus client...\n")
    for corpus_name in corpus_list:
        print(f"Processing corpus: {corpus_name}")
        chunk_numbers = len(os.listdir(f"/data/jiwoong/workspace/milvus/corpus/{corpus_name}/data/"))
        for chunk_num in range(chunk_numbers):
            vectors = np.load(f"/data/jiwoong/workspace/milvus/corpus/{corpus_name}/vector/{corpus_name}_embeds_{chunk_num}.npy")
            with open(f"/data/jiwoong/workspace/milvus/corpus/{corpus_name}/json/{corpus_name}_chunk_{chunk_num}.json", 'r') as jsfile:
                texts = json.load(jsfile)
            data_loaded = [
                    {"vector": vectors[ith].tolist(), "text": texts[ith]['contents_chunk'], "source": corpus_name}
                    for ith in range(0, len(vectors))
                ]
            for i in range(0, len(data_loaded), batch_size):
                batch = data_loaded[i:i+batch_size]
                for idx, item in enumerate(batch):
                    item['id'] = global_offset + idx
                client.insert(collection_name=collection_name, data=batch)
                global_offset += len(batch)
        print(f"{corpus_name} Uploaded!")

In [ ]:
data_upload(client)

data uploading to 'rag2' collection in milvus client...

Processing corpus: statpearls
statpearls Uploaded!
Processing corpus: recop
recop Uploaded!
Processing corpus: textbooks
textbooks Uploaded!
Processing corpus: cpg


In [ ]:
client.create_collection(
    collection_name="medagents",
    dimension=768,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)

In [ ]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530"
)

def retrieve(query, source_filter , client, topk = 100):
    search_res = client.search(
    collection_name='rag2',
    data=[
        medcpt_query_embedding_function(query)
    ],  
    limit=topk,  
    search_params={"metric_type": "IP", "params": {}},  
    output_fields=["text", 'source'],  
    filter = source_filter 
    )
    
    evidence_list = [result["entity"]["text"] for result in search_res[0][:topk]]
    return evidence_list

def retrieve_filtered_sources(query, client, allowed_sources = ["source == 'PubMed'", "source == 'PMC'", "source == 'Textbook'", "source == 'CPG'", "source == 'statpearls'", "source == 'recop'", "source == 'textbooks'", "source == 'cpg'"], topk=100):
#def retrieve_filtered_sources(query, client, allowed_sources = ["source == 'statpearls'", "source == 'recop'", "source == 'textbooks'", "source == 'cpg'"], topk=100):
    evidence_list = []
    for source_filter  in allowed_sources:
        evidence_list.extend(retrieve(query, source_filter, client, topk=topk, ))
    return evidence_list


device = "cuda:1"
model_q = AutoModel.from_pretrained("ncbi/MedCPT-Query-Encoder").to(device)
tokenizer_q = AutoTokenizer.from_pretrained("ncbi/MedCPT-Query-Encoder")

def medcpt_query_embedding_function(docs):
    encoded = tokenizer_q(
        docs,
        truncation=True,
        padding=True,
        return_tensors='pt',
        max_length=512,
    )
    encoded = {k: v.to(device) for k, v in encoded.items()}
    with torch.no_grad():
        embeds = model_q(**encoded).last_hidden_state[:, 0, :]
    embeds = embeds.cpu().numpy()
    return embeds[0].tolist()

model_c = AutoModelForSequenceClassification.from_pretrained("ncbi/MedCPT-Cross-Encoder").to(device)
tokenizer_c = AutoTokenizer.from_pretrained("ncbi/MedCPT-Cross-Encoder")

def rerank(query, doc_list):
    pairs = [[query, doc] for doc in doc_list]
    with torch.no_grad():
        encoded = tokenizer_c(
            pairs,
            truncation=True,
            padding=True,
            return_tensors="pt",
            max_length=512,
        )
        encoded = {k: v.to(device) for k, v in encoded.items()} 
        logits = model_c(**encoded).logits.squeeze(dim=1).detach().cpu()
    sorted_indices = torch.argsort(logits, descending=True)
    ranked_docs = [doc_list[i] for i in sorted_indices]
    return ranked_docs

def retrieve_meta(query, client, topk = 100):
    search_res = client.search(
    collection_name='medagents',
    data=[
        medcpt_query_embedding_function('A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?')
    ],  
    limit=topk,  
    search_params={"metric_type": "IP", "params": {}},  
    output_fields=["text", 'source'],  
    #filter = "source == 'statpearls'"
    )
    
    evidene_list = []
    for kth in tqdm(range(topk)):
        corpus, chunk_num, stacked_id, = search_res[0][kth]['entity']['text'].split("_")
        print(corpus, chunk_num, stacked_id)
        with open(f"/data/jiwoong/workspace/milvus/corpus/{corpus}/json/{corpus}_chunk_{chunk_num}.json", 'r') as jsfile:
            evidene_list.append((json.load(jsfile)[int(stacked_id)]['contents_chunk']))
            
    return evidene_list

In [41]:
import json
medqa_test_path = "/data/jiwoong/workspace/MedAgents/datasets/MedQA/50_sampled_hard_medqa/test.jsonl"
medqa_test = [] 
with open(medqa_test_path, 'r') as file:
    for line in file:
        medqa_test.append(json.loads(line))

In [ ]:
from openai import AzureOpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
from dotenv import load_dotenv

azure client = AzureOpenAI(
    azure_endpoint = "https://azure-openai-miblab-ncu.openai.azure.com/",
    api_key = os.getenv("azure_api_key"),
    api_version = "2024-08-01-preview",
)


def generate_response(question):
    response = azure_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=question,
        logprobs=True,
        temperature=0,
        n=1
    )
    return response.choices[0].message.content

def generate_responses_in_parallel(prompts):
    result = [None] * len(prompts)  # Initialize list with None to store results in order
    # Function to generate response and store it at the correct index
    def generate_response_with_index(index, prompt):
        response = generate_response(prompt)
        return index, response
    with ThreadPoolExecutor() as executor:
        # Submit all tasks to the executor and keep track of the future objects
        futures = {executor.submit(generate_response_with_index, i, prompt): i for i, prompt in enumerate(prompts)}

        for future in tqdm(as_completed(futures), total=len(prompts)):
            index, response = future.result()  # Get the result when a thread completes
            result[index] = response  # Store the response in the correct index
    return result

In [74]:
def encode_question_gen_query2doc(question: str,) -> list[str]:
    prompts= []
    few_shot_template = """...
Example:

Query: A 39-year-old woman presents to the family medicine clinic to be evaluated by her physician for weight gain. She reports feeling fatigued most of the day despite eating a healthy diet and exercising regularly. The patient smokes a half-pack of cigarettes daily and has done so for the last 23 years. She is employed as a phlebotomist by the Red Cross. She has a history of hyperlipidemia for which she takes atorvastatin. She is unaware of her vaccination history, and there is no documented record of her receiving any vaccinations. Her heart rate is 76/min, respiratory rate is 14/min, temperature is 37.3°C (99.1°F), body mass index (BMI) is 33 kg/m2, and blood pressure is 128/78 mm Hg. The patient appears alert and oriented. Lung and heart auscultation are without audible abnormalities. The physician orders a thyroid panel to determine if that patient has hypothyroidism. Which of the following recommendations may be appropriate for the patient at this time? A) Hepatitis B vaccination B) Low-dose chest CT C) Hepatitis C vaccination D) Shingles vaccination
Passage: against vaccine-preventable diseases. Every visit by an adult to a health-care provider should be an opportunity to provide this protection. Several factors need to be con sidered before any patient is vaccinated. These include the susceptibility of the patient, the risk of exposure to the disease, the risk from the disease, and the benefits and risks from the immunizing agent. Physicians should maintain detailed information about previous vaccina tions received by each individual, including type of vaccination, date of receipt, and adverse events, if any, following vaccination. Information should also include the person's history of vaccine-preventable illnesses, occupation, and lifestyle. Vaccine histories ideally should be based on written documentation to ascertain whether vaccines and toxoids were administered at appropriate ages and at proper intervals. Close attention to factors

Query: A 23-year-old male presents to his primary care physician after an injury during a rugby game. The patient states that he was tackled and ever since then has had pain in his knee. The patient has tried NSAIDs and ice to no avail. The patient has no past medical history and is currently taking a multivitamin, fish oil, and a whey protein supplement. On physical exam you note a knee that is heavily bruised. It is painful for the patient to bear weight on the knee, and passive motion of the knee elicits some pain. There is laxity at the knee to varus stress. The patient is wondering when he can return to athletics. Which of the following is the most likely diagnosis? A) Medial collateral ligament tear B) Lateral collateral ligament tear C) Anterior cruciate ligament tear D) Posterior cruciate ligament tear
Passage: Diagnosing PCL Injuries: History, Physical Examination, Imaging Studies, Arthroscopic Evaluation. Isolated posterior cruciate ligament (PCL) injuries are uncommon and can be easily missed with physical examination. The purpose of this article is to give an overview of the clinical, diagnostic and arthroscopic evaluation of a PCL injured knee. There are some specific injury mechanisms that can cause a PCL including the dashboard direct anterior blow and hyperflexion mechanisms. During the diagnostic process it is important to distinguish between an isolated or multiligament injury and whether the problem is acute or chronic. Physical examination can be difficult in an acutely injured knee because of pain and swelling, but there are specific functional tests that can indicate a PCL tear. Standard x-ray's and stress views are very useful imaging modalities

Query: A 45-year-old woman is in a high-speed motor vehicle accident and suffers multiple injuries to her extremities and abdomen. In the field, she was bleeding profusely bleeding and, upon arrival to the emergency department, she is lethargic and unable to speak. Her blood pressure on presentation is 70/40 mmHg. The trauma surgery team recommends emergency exploratory laparotomy. While the patient is in the trauma bay, her husband calls and says that the patient is a Jehovah's witness and that her religion does not permit her to receive a blood transfusion. No advanced directives are available. Which of the following is an appropriate next step? A) Provide transfusions as needed B) Withhold transfusion based on husband's request C) Obtain an ethics consult D) Obtain a court order for transfusion
Passage: Legal and ethical issues in safe blood transfusion. This is another D and C Act requirement which is seldom followed, possibly because there are no standard guidelines.

Query: A 4-year-old male is accompanied by his mother to the pediatrician. His mother reports that over the past two weeks, the child has had intermittent low grade fevers and has been more lethargic than usual. The child’s past medical history is notable for myelomeningocele complicated by lower extremity weakness as well as bowel and bladder dysfunction. He has been hospitalized multiple times at an outside facility for recurrent urinary tract infections. The child is in the 15th percentile for both height and weight. His temperature is 100.7°F (38.2°C), blood pressure is 115/70 mmHg, pulse is 115/min, and respirations are 20/min. Physical examination is notable for costovertebral angle tenderness that is worse on the right. Which of the following would most likely be found on biopsy of this patient’s kidney? A) Mononuclear and eosinophilic infiltrate B) Replacement of renal parenchyma with foamy histiocytes C) Destruction of the proximal tubule and medullary thick ascending limb D) Tubular colloid casts with diffuse lymphoplasmacytic infiltrate
Passage: The natural history of urinary infection in adults. The vast majority of otherwise healthy adults with anatomically and functionally normal urinary tracts experience few untoward long-term consequences from symptomatic or asymptomatic UTIs. Effective early treatment of symptomatic infection rapidly curtails bacterial invasion and the resulting inflammatory response. Rarely, uncomplicated acute pyelonephritis causes suppuration and renal scarring. Urinary infections in patients with renal calculi, obstructed urinary tract, neurogenic bladder, or diabetes are frequently much more destructive and have ongoing sequelae. Strategies to treat both the infection and the complications are often necessary to alter this outcome.
...

Query: {}
Passage:
""".format(question)
    prompts.append({"role": "system", "content": "Write a medical passage that can help answer the given query. Include key information or terminology for the answer."})
    prompts.append({"role": "user", "content": few_shot_template})
    return prompts

def encode_question_gen(evidence: str,question: str,options: str,) -> list[str]:
    prompts= []
    prompt_rag = """Here is the relevant document:
{}

Here is the question:
{}

Here are the options:
{}

Please think step-by-step and generate your output.
""".format(evidence, question, options)
    prompts.append({"role": "system", "content": "The following is a multiple-choice question about medical knowledge. Solve this in a step-by-step fashion, starting by summarizing the available information. Output a single option from the given options as the final answer. You are strongly required to follow the specified output format; conclude your response with the phrase \"the answer is ([option_index]) [answer_string]\"."})
    prompts.append({"role": "user", "content": prompt_rag})
    return prompts

In [10]:
len(medqa_test)

50

In [75]:
chatgpt_prompts = []
for index, test in enumerate(medqa_test):
    chatgpt_prompts.append(encode_question_gen_query2doc(test['question'] + ' (A) ' + test['options']['A'] + ' (B) ' + test['options']['B'] + ' (C) ' + test['options']['C'] + ' (D) ' + test['options']['D'], ))

In [13]:
pseudodocs = generate_responses_in_parallel(chatgpt_prompts)

100%|██████████| 50/50 [00:11<00:00,  4.40it/s]


In [54]:
with open("/data/jiwoong/retriever/input/query2doc/hard_medqa_gpt4omini_query2doc_fewshot", 'r') as jsfile:
    pseudodocs = json.load(jsfile)

In [68]:
realdocs = []
for pseudodoc in tqdm(pseudodocs):
    realdocs.append(retrieve_filtered_sources(pseudodoc, client, allowed_sources = ["source == 'PubMed'", "source == 'PMC'", "source == 'Textbook'", "source == 'CPG'"], topk=100))

100%|██████████| 50/50 [03:49<00:00,  4.60s/it]


In [70]:
rankeddocs = []
for index, realdoc in tqdm(enumerate(realdocs)):
    test = medqa_test[index]
    rankeddocs.append(rerank(
        pseudodocs[index],
#        test['question'] + ' (A) ' + test['options']['A'] + ' (B) ' + test['options']['B'] + ' (C) ' + test['options']['C'] + ' (D) ' + test['options']['D'],
        realdoc))

50it [01:38,  1.97s/it]


In [77]:
chatgpt_prompts = []
for index, test in enumerate(medqa_test):
    context = "\n".join(rankeddocs[index][:100])
    chatgpt_prompts.append(encode_question_gen(context, test['question'], '(A) ' + test['options']['A'] + ' (B) ' + test['options']['B'] + ' (C) ' + test['options']['C'] + ' (D) ' + test['options']['D'], ))

result_100 = generate_responses_in_parallel(chatgpt_prompts)
temp = [0]*50
for index, answer in enumerate(result_100):
    last_sentence = answer.split('answer is')[-1]
    matches = re.findall(r"\([A-Da-d]\)", last_sentence)
    chosen_option_2 = ""
    if matches:
        chosen_option_2 = matches[-1][1]
    if medqa_test[index]['answer_idx'] == chosen_option_2:
        temp[index] = 1
    else:
        temp[index] = 0
print(sum(temp) / len(temp))

100%|██████████| 50/50 [00:14<00:00,  3.34it/s]

0.5


0.44


In [18]:
chatgpt_prompts = []
for index, test in enumerate(hard_medqa):
    context = "\n".join(rankeddoc[index][:25])
    chatgpt_prompts.append(encode_question_gen(context, test['question'], '(A) ' + test['options']['A'] + ' (B) ' + test['options']['B'] + ' (C) ' + test['options']['C'] + ' (D) ' + test['options']['D'], ))

result_100 = generate_responses_in_parallel(chatgpt_prompts)

temp = [0]*50
for index, answer in enumerate(result_100):
    last_sentence = answer.split('answer is')[-1]
    matches = re.findall(r"\([A-Da-d]\)", last_sentence)
    chosen_option_2 = ""
    if matches:
        chosen_option_2 = matches[-1][1]
    if hard_medqa[index]['answer_idx'] == chosen_option_2:
        temp[index] = 1
    else:
        temp[index] = 0
print(sum(temp) / len(temp))

100%|██████████| 50/50 [00:12<00:00,  4.16it/s]

0.54


In [ ]:
with open("/data/jiwoong/retriever/output/query2doc/evidence_hard_medqa_gpt4omini_query2doc_fewshot.json", 'r') as jsfile:
    evidence_hard_medqa_gpt4omini_query2doc_fewshot = json.load(jsfile)

## original

In [7]:
with open("/data/jiwoong/retriever/output/query2doc/medqa__gpt4omini_query2doc_combined.json", 'r') as jsfile:
    medqa__gpt4omini_query2doc_combined = json.load(jsfile)

In [6]:
hard_medqa = []
with open("/data/jiwoong/workspace/MedAgents/datasets/MedQA/50_sampled_hard_medqa/test.jsonl", 'r') as jsfile:
    for line in jsfile:
        hard_medqa.append(json.loads(line))

In [10]:
chatgpt_prompts = []
for index, test in enumerate(hard_medqa):
    context = ""
    for ith in range(25):
        context+=medqa__gpt4omini_query2doc_combined[index][ith]
        context+="\n"
    chatgpt_prompts.append(encode_question_gen(context, test['question'], '(A) ' + test['options']['A'] + ' (B) ' + test['options']['B'] + ' (C) ' + test['options']['C'] + ' (D) ' + test['options']['D'], ))

result = generate_responses_in_parallel(chatgpt_prompts)

temp = [0]*50
for index, answer in enumerate(result):
    last_sentence = answer.split('answer is')[-1]
    matches = re.findall(r"\([A-D]\)", last_sentence)
    chosen_option_2 = ""
    if matches:
        chosen_option_2 = matches[0][1]
    if hard_medqa[index]['answer_idx'] == chosen_option_2:
        temp[index] = 1
    else:
        temp[index] = 0
print(sum(temp) / len(temp))

100%|██████████| 50/50 [00:11<00:00,  4.29it/s]

0.44


In [95]:
chatgpt_prompts = []
for index, test in enumerate(medqa_test):
    context = ""
    for ith in range(100):
        context+=medqa__gpt4omini_query2doc_combined[index+250][ith]
        context+="\n"
    chatgpt_prompts.append(encode_question_gen(context, test['question'], '(A) ' + test['options']['A'] + ' (B) ' + test['options']['B'] + ' (C) ' + test['options']['C'] + ' (D) ' + test['options']['D'], ))

result = generate_responses_in_parallel(chatgpt_prompts)

temp = [0]*50
for index, answer in enumerate(result):
    last_sentence = answer.split('answer is')[-1]
    matches = re.findall(r"\([A-D]\)", last_sentence)
    chosen_option_2 = ""
    if matches:
        chosen_option_2 = matches[0][1]
    if medqa_test[index]['answer_idx'] == chosen_option_2:
        temp[index] = 1
    else:
        temp[index] = 0
print(sum(temp) / len(temp))

100%|██████████| 50/50 [00:14<00:00,  3.48it/s]

0.84


In [116]:
chatgpt_prompts = []
for index, test in enumerate(medqa_test):
    context = ""
    for ith in range(100):
        context+=medqa__gpt4omini_query2doc_combined[index][ith]
        context+="\n"
    chatgpt_prompts.append(encode_question_gen(context, test['question'], '(A) ' + test['options']['A'] + ' (B) ' + test['options']['B'] + ' (C) ' + test['options']['C'] + ' (D) ' + test['options']['D'], ))

result = generate_responses_in_parallel(chatgpt_prompts)

temp = [0]*50
for index, answer in enumerate(result):
    last_sentence = answer.split('answer is')[-1]
    matches = re.findall(r"\([A-D]\)", last_sentence)
    chosen_option_2 = ""
    if matches:
        chosen_option_2 = matches[0][1]
    if medqa_test[index]['answer_idx'] == chosen_option_2:
        temp[index] = 1
    else:
        temp[index] = 0
print(sum(temp) / len(temp))

100%|██████████| 50/50 [00:12<00:00,  3.88it/s]

0.84


In [108]:
chatgpt_prompts = []
for index, test in enumerate(medqa_test):
    context = ""
    for ith in range(25):
        context+=rankeddoc[index][ith]
        context+="\n"
    chatgpt_prompts.append(encode_question_gen(context, test['question'], '(A) ' + test['options']['A'] + ' (B) ' + test['options']['B'] + ' (C) ' + test['options']['C'] + ' (D) ' + test['options']['D'], ))

In [109]:
result_1 = generate_responses_in_parallel(chatgpt_prompts)

100%|██████████| 50/50 [00:09<00:00,  5.42it/s]


In [110]:
correctedness = {}
temp = [0]*50
for index, answer in enumerate(result_1):
    last_sentence = answer.split('answer is')[-1]
    matches = re.findall(r"\([A-D]\)", last_sentence)
    chosen_option_2 = ""
    if matches:
        chosen_option_2 = matches[0][1]
    if medqa_test[index]['answer_idx'] == chosen_option_2:
        temp[index] = 1
    else:
        temp[index] = 0
print(sum(temp) / len(temp))
correctedness["reranked_16"] = temp 

0.82


In [111]:
print(result_1[13])

To determine the pattern of inheritance for the disorder presented in the case of the 15-year-old boy, we can analyze the information provided:

1. **Patient Presentation**: The boy is experiencing extreme fatigue and shortness of breath, with a significantly low hemoglobin level (5 g/dL) and an enlarged spleen. These symptoms suggest a form of anemia.

2. **Family History**: The father and aunt both have mild anemia, indicating a potential hereditary component to the condition.

3. **Recovery**: The patient recovers over two weeks, with hemoglobin levels rising to 11 g/dL, which suggests that the anemia may be acute and possibly related to a transient condition rather than a chronic genetic disorder.

4. **Inheritance Patterns**:
   - **Autosomal Dominant**: Typically, this would show a clear pattern of inheritance in every generation, which is not indicated here.
   - **X-linked Recessive**: This would primarily affect males, and females would be carriers. The family history does not

In [104]:
print(result_1[13])

To determine the pattern of inheritance for the patient's underlying disorder, we can analyze the information provided:

1. **Patient's Symptoms**: The 15-year-old boy presents with extreme fatigue, shortness of breath, pallor, and splenomegaly. His hemoglobin level is critically low at 5 g/dL, indicating a severe anemia.

2. **Family History**: The father and aunt both suffer from mild anemia, suggesting a hereditary component to the condition.

3. **Recovery**: The patient recovers over two weeks, with hemoglobin rising to 11 g/dL, which indicates that the anemia may be acute and possibly related to a transient condition rather than a chronic genetic disorder.

4. **Peripheral Blood Smear**: While the specific details of the blood smear are not provided, the context suggests that it may show features consistent with a hereditary anemia, such as hereditary spherocytosis or another type of hemolytic anemia.

5. **Inheritance Patterns**:
   - **Autosomal Dominant**: Typically, this woul

In [99]:
print(result_1[13])

To determine the pattern of inheritance for the disorder presented in the case of the 15-year-old boy, we can analyze the information provided:

1. **Patient Presentation**: The boy is experiencing extreme fatigue and shortness of breath, with a hemoglobin level of 5 g/dL, indicating severe anemia. His spleen is enlarged, which can be associated with certain types of anemia.

2. **Family History**: The father and aunt both suffer from mild anemia, suggesting a hereditary component to the condition.

3. **Recovery**: The patient recovers over two weeks, with his hemoglobin rising to 11 g/dL, indicating that the anemia may be acute and possibly related to a transient condition rather than a chronic genetic disorder.

4. **Types of Inheritance**:
   - **Autosomal Dominant**: Typically, this would show a clear pattern in multiple generations, which is not strongly indicated here.
   - **X-linked Recessive**: This usually affects males more severely and is passed from carrier mothers to aff

In [113]:
context = ""
for i in range(25):
    context += rankeddoc[13][i]
    context += '\n'
print(context)

Causes of Sickle Cell Disease. SCD is caused by mutations in the hemoglobin beta (HBB) gene and is inherited as an autosomal recessive trait. Genetic diseases are determined by two genes, one received from the father and one from the mother. Recessive genetic disorders occur when an individual inherits the same altered gene for the same trait from each parent. If an individual receives one normal gene and one gene for the disease, the person will be a carrier for the disease, but usually will not show symptoms
InternalMed_Harrison. On the other hand, allelic variants may reflect mutations that clearly alter the function of a gene product. The common Glu6Val (E6V) sickle cell mutation in the β-globin gene and the ΔF508 deletion of phenylalanine (F) in the CFTR gene are examples of allelic variants of these genes that result in disease. Because each individual has two copies of each chromosome (one inherited from the mother and one inherited from the father), he or she can have only two 

In [98]:
for ith, i in enumerate(temp):
    if i == 0:
        print(ith)


3
4
9
11
13
15
30
31
38
43
46


In [127]:
for ith, i in enumerate(temp):
    if i == 0:
        print(ith)


1
3
9
13
15
30
31
34
38
43
44
46


In [117]:
for ith, i in enumerate(temp):
    if i == 0:
        print(ith)


13
15
30
38
43
46


In [114]:
print(chatgpt_prompts[13][1]['content'])

Here is the relevant document:
Causes of Sickle Cell Disease. SCD is caused by mutations in the hemoglobin beta (HBB) gene and is inherited as an autosomal recessive trait. Genetic diseases are determined by two genes, one received from the father and one from the mother. Recessive genetic disorders occur when an individual inherits the same altered gene for the same trait from each parent. If an individual receives one normal gene and one gene for the disease, the person will be a carrier for the disease, but usually will not show symptoms
InternalMed_Harrison. On the other hand, allelic variants may reflect mutations that clearly alter the function of a gene product. The common Glu6Val (E6V) sickle cell mutation in the β-globin gene and the ΔF508 deletion of phenylalanine (F) in the CFTR gene are examples of allelic variants of these genes that result in disease. Because each individual has two copies of each chromosome (one inherited from the mother and one inherited from the father

In [44]:
print(chatgpt_prompts[10][1]['content'])

Here is the relevant document:
clinical practice guidelines. acquiring HSV-2); - importance of abstaining from sexual activity with uninfected partners when lesions or prodromal symptoms are present; - effectiveness of daily use of valacyclovir in reducing risk for transmission of HSV-2, and the lack of effectiveness of episodic or suppressive therapy in persons with HIV and HSV infection in reducing risk for transmission to partners who might be at risk for HSV-2 acquisition; - effectiveness of male latex condoms, which when used consistently and correctly can
clinical practice guidelines. provider (for a patient reporting a history of or presenting with an atypical case, a mild case, or both, health-care providers should seek either an epidemiologic link to a typical varicella case or to a laboratory-confirmed case or evidence of laboratory confirmation, if it was performed at the time of acute disease); 4) history of herpes zoster based on health-care provider diagnosis or verificat

In [40]:
for ith, i in enumerate(temp):
    if i == 0:
        print(ith)


3
9
10
11
13
15
22
29
30
31
38
43
46


In [15]:
res = client.search(
    collection_name="demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)


res = client.search(
    collection_name="demo_collection",
    data=embedding_fn.encode_queries(["tell me AI related information"]),
    filter="subject == 'biology'",
    limit=2,
    output_fields=["text", "subject"],
)

data: ["[{'id': 5, 'distance': 0.05801853910088539, 'entity': {'text': 'DDR1 is involved in cancers and fibrosis.', 'subject': 'biology'}}]"] 
